# Validation model
Test and experiments to check the performance of one or more models

In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
from keras import layers, losses
from keras.models import Model
from shapely.geometry import Polygon, box

from tqdm.keras import TqdmCallback
from sklearn.metrics import confusion_matrix

import rasterio
from rasterio import plot as rastPlt
from rasterio.plot import reshape_as_raster
from rasterio.merge import merge as rasterMerge
from rasterio.mask import mask as rasterMask


In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS.tif"
pathValidation = folderName + "Population_GHSL.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)
print(len(dataCoords))

#### Validation

In [ ]:
with rasterio.open(pathValidation) as f:
  sampleTile, _ = rasterMask(f, [Polygon(dataCoords[1000])], crop=True) # arbitrary coordinate for a sample
print('Validation shape:',sampleTile.shape) # shape sample

print('Process validation...')
# If any of the pixels have a settlement then true else false
getValid = lambda data : [ int(250/255 < img.max()) for img in data ]
resultValid = utils.scanSatellite(pathValidation, dataCoords, getValid, batch=1000, res=sampleTile.shape[1])

In [ ]:
model1 = keras.models.load_model('model/cnn_64px_32_16_8_v2')
model2 = keras.models.load_model('model/cnn_64px_32_16_8_v3')
model3 = keras.models.load_model('model/cnn_64px_32_16_8_v4')

In [ ]:
print('Process score...')
getScore = lambda data : model1.predict(data, verbose=0).flatten()
result1 = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=100, res=64)

In [ ]:
print('Process score...')
getScore = lambda data : model2.predict(data, verbose=0).flatten()
result2 = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=100, res=64)

In [ ]:
print('Process score...')
getScore = lambda data : model3.predict(data, verbose=0).flatten()
result3 = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=100, res=64)

In [ ]:
threshold = 0.5

print('Process confustion matrix...')
resultTest1 = (np.asarray(result1) > threshold).astype(int)
confusionMatrix = confusion_matrix(resultValid, resultTest1)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

In [ ]:
threshold = 0.04

print('Process confustion matrix...')
resultTest2 = (np.asarray(result2) > threshold).astype(int)
confusionMatrix = confusion_matrix(resultValid, resultTest2)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

In [ ]:
threshold = 0.05

print('Process confustion matrix...')
resultTest3 = (np.asarray(result3) > threshold).astype(int)
confusionMatrix = confusion_matrix(resultValid, resultTest3)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

In [ ]:
fig, axs = matPlt.subplots(1,4, dpi=240)
with rasterio.open(pathSatellite) as s: utils.displayTiles([s.read()], [s.transform],axs[0])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())

axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())

axs[3].set_xlim(axs[0].get_xlim())
axs[3].set_ylim(axs[0].get_ylim())

#with rasterio.open(pathValidation) as p: rastPlt.show(p, ax=axs[1])

resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, resultTest1, resultValid)
utils.displayTiles([resultImg], [resultMeta], axs[1])
resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, resultTest2, resultValid)
utils.displayTiles([resultImg], [resultMeta], axs[2])
resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, resultTest3, resultValid)
utils.displayTiles([resultImg], [resultMeta], axs[3])
